In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


In [2]:
# Import Libraries 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.datasets
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn import metrics

In [3]:
train_data = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')

In [4]:
train_data.head(1)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500


In [5]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

* ``SalePrice`` : the property's sale price in dollars. This is the target variable that you're trying to predict.
* ``MSSubClass`` : The building class
* ``MSZoning`` : The general zoning classification
* ``LotFrontage`` : Linear feet of street connected to property
* ``LotArea`` : Lot size in square feet
* ``Street`` : Type of road access
* ``Alley`` : Type of alley access
* ``LotShape`` : General shape of property
* ``LandContour`` : Flatness of the property
* ``Utilities`` : Type of utilities available
* ``LotConfig`` : Lot configuration
* ``LandSlope`` : Slope of property
* ``Neighborhood`` : Physical locations within Ames city limits
* ``Condition1`` : Proximity to main road or railroad
* ``Condition2`` : Proximity to main road or railroad (if a second is present)
* ``BldgType`` : Type of dwelling
* ``HouseStyle`` : Style of dwelling
* ``OverallQual`` : Overall material and finish quality
* ``OverallCond`` : Overall condition rating
* ``YearBuilt`` : Original construction date
* ``YearRemodAdd`` : Remodel date
* ``RoofStyle`` : Type of roof
* ``RoofMatl`` : Roof material
* ``Exterior1st`` : Exterior covering on house
* ``Exterior2nd`` : Exterior covering on house (if more than one material)
* ``MasVnrType`` : Masonry veneer type
* ``MasVnrArea`` : Masonry veneer area in square feet
* ``ExterQual`` : Exterior material quality
* ``ExterCond`` : Present condition of the material on the exterior
* ``Foundation`` : Type of foundation
* ``BsmtQual`` : Height of the basement
* ``BsmtCond``: General condition of the basement
* ``BsmtExposure``: Walkout or garden level basement walls
* ``BsmtFinType1``: Quality of basement finished area
* ``BsmtFinSF1``: Type 1 finished square feet
* ``BsmtFinType2``: Quality of second finished area (if present)
* ``BsmtFinSF2``: Type 2 finished square feet
* ``BsmtUnfSF``: Unfinished square feet of basement area
* ``TotalBsmtSF``: Total square feet of basement area
* ``Heating``: Type of heating
* ``HeatingQC``: Heating quality and condition
* ``CentralAir``: Central air conditioning
* ``Electrical``: Electrical system
* ``1stFlrSF``: First Floor square feet
* ``2ndFlrSF``: Second floor square feet
* ``LowQualFinSF``: Low quality finished square feet (all floors)
* ``GrLivArea``: Above grade (ground) living area square feet
* ``BsmtFullBath``: Basement full bathrooms
* ``BsmtHalfBath``: Basement half bathrooms
* ``FullBath``: Full bathrooms above grade
* ``HalfBath``: Half baths above grade
* ``Bedroom``: Number of bedrooms above basement level
* ``Kitchen``: Number of kitchens
* ``KitchenQual``: Kitchen quality
* ``TotRmsAbvGrd``: Total rooms above grade (does not include bathrooms)
* ``Functional``: Home functionality rating
* ``Fireplaces``: Number of fireplaces
* ``FireplaceQu``: Fireplace quality
* ``GarageType``: Garage location
* ``GarageYrBlt``: Year garage was built
* ``GarageFinish``: Interior finish of the garage
* ``GarageCars``: Size of garage in car capacity
* ``GarageArea``: Size of garage in square feet
* ``GarageQual``: Garage quality
* ``GarageCond``: Garage condition
* ``PavedDrive``: Paved driveway
* ``WoodDeckSF``: Wood deck area in square feet
* ``OpenPorchSF``: Open porch area in square feet
* ``EnclosedPorch``: Enclosed porch area in square feet
* ``3SsnPorch``: Three season porch area in square feet
* ``ScreenPorch``: Screen porch area in square feet
* ``PoolArea``: Pool area in square feet
* ``PoolQC``: Pool quality
* ``Fence``: Fence quality
* ``MiscFeature``: Miscellaneous feature not covered in other categories
* ``MiscVal``: Value of miscellaneous feature
* ``MoSold``: Month Sold
* ``YrSold``: Year Sold
* ``SaleType``: Type of sale
* ``SaleCondition``: Condition of sale

In [6]:
cat_col = []
for col in train_data.select_dtypes('object'):
    print('==============================')
    display(train_data[col].value_counts())
    cat_col.append(col)
    print('==============================')
    

RL         1151
RM          218
FV           65
RH           16
C (all)      10
Name: MSZoning, dtype: int64

Pave    1454
Grvl       6
Name: Street, dtype: int64

Grvl    50
Pave    41
Name: Alley, dtype: int64

Reg    925
IR1    484
IR2     41
IR3     10
Name: LotShape, dtype: int64

Lvl    1311
Bnk      63
HLS      50
Low      36
Name: LandContour, dtype: int64

...

Gd    3
Ex    2
Fa    2
Name: PoolQC, dtype: int64

MnPrv    157
GdPrv     59
GdWo      54
MnWw      11
Name: Fence, dtype: int64

Shed    49
Gar2     2
Othr     2
TenC     1
Name: MiscFeature, dtype: int64

WD       1267
New       122
COD        43
ConLD       9
ConLI       5
ConLw       5
CWD         4
Oth         3
Con         2
Name: SaleType, dtype: int64

Normal     1198
Partial     125
Abnorml     101
Family       20
Alloca       12
AdjLand       4
Name: SaleCondition, dtype: int64

In [7]:
cat_col

['MSZoning',
 'Street',
 'Alley',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 'KitchenQual',
 'Functional',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'PoolQC',
 'Fence',
 'MiscFeature',
 'SaleType',
 'SaleCondition']

In [8]:
from sklearn.preprocessing import LabelEncoder
Le = LabelEncoder()
train_data[cat_col] = Le.fit_transform(cat_col)

In [9]:
train_data

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,31,65.0,8450,41,0,30,27,42,...,0,36,15,33,0,2,2008,40,39,208500
1,2,20,31,80.0,9600,41,0,30,27,42,...,0,36,15,33,0,5,2007,40,39,181500
2,3,60,31,68.0,11250,41,0,30,27,42,...,0,36,15,33,0,9,2008,40,39,223500
3,4,70,31,60.0,9550,41,0,30,27,42,...,0,36,15,33,0,2,2006,40,39,140000
4,5,60,31,84.0,14260,41,0,30,27,42,...,0,36,15,33,0,12,2008,40,39,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,31,62.0,7917,41,0,30,27,42,...,0,36,15,33,0,8,2007,40,39,175000
1456,1457,20,31,85.0,13175,41,0,30,27,42,...,0,36,15,33,0,2,2010,40,39,210000
1457,1458,70,31,66.0,9042,41,0,30,27,42,...,0,36,15,33,2500,5,2010,40,39,266500
1458,1459,20,31,68.0,9717,41,0,30,27,42,...,0,36,15,33,0,4,2010,40,39,142125


In [10]:
train_data.head(3)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,31,65.0,8450,41,0,30,27,42,...,0,36,15,33,0,2,2008,40,39,208500
1,2,20,31,80.0,9600,41,0,30,27,42,...,0,36,15,33,0,5,2007,40,39,181500
2,3,60,31,68.0,11250,41,0,30,27,42,...,0,36,15,33,0,9,2008,40,39,223500


In [11]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   int64  
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   int64  
 6   Alley          1460 non-null   int64  
 7   LotShape       1460 non-null   int64  
 8   LandContour    1460 non-null   int64  
 9   Utilities      1460 non-null   int64  
 10  LotConfig      1460 non-null   int64  
 11  LandSlope      1460 non-null   int64  
 12  Neighborhood   1460 non-null   int64  
 13  Condition1     1460 non-null   int64  
 14  Condition2     1460 non-null   int64  
 15  BldgType       1460 non-null   int64  
 16  HouseStyle     1460 non-null   int64  
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [12]:
train_data.shape

(1460, 81)

In [13]:
X = train_data.drop(['SalePrice','Id'],axis=1)
Y = train_data['SalePrice']
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.3,train_size=0.7)

In [14]:
XG = XGBRegressor()
XG.fit(x_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, ...)

In [15]:
XG.score(x_test,y_test)

0.838678559557971

In [16]:
test_data = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')

In [17]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1459 non-null   int64  
 1   MSSubClass     1459 non-null   int64  
 2   MSZoning       1455 non-null   object 
 3   LotFrontage    1232 non-null   float64
 4   LotArea        1459 non-null   int64  
 5   Street         1459 non-null   object 
 6   Alley          107 non-null    object 
 7   LotShape       1459 non-null   object 
 8   LandContour    1459 non-null   object 
 9   Utilities      1457 non-null   object 
 10  LotConfig      1459 non-null   object 
 11  LandSlope      1459 non-null   object 
 12  Neighborhood   1459 non-null   object 
 13  Condition1     1459 non-null   object 
 14  Condition2     1459 non-null   object 
 15  BldgType       1459 non-null   object 
 16  HouseStyle     1459 non-null   object 
 17  OverallQual    1459 non-null   int64  
 18  OverallC

In [18]:
from sklearn.preprocessing import LabelEncoder
Le = LabelEncoder()
test_data[cat_col] = Le.fit_transform(cat_col)

In [19]:
y_prediction = XG.predict(test_data.drop('Id',axis=1))

In [20]:
test_data['Prediction'] = y_prediction

In [21]:
submission_data = pd.DataFrame()

In [22]:
submission_data['Id'] = test_data['Id']
submission_data['SalePrice'] = test_data['Prediction']

In [23]:
submission_data.to_csv('submission.csv')